---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix
from datetime import date
from sklearn.ensemble import RandomForestClassifier

def blight_model():
    
    # Test Train Data------------------------------------------------------------------------------------------------------- 
    train_data = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    test_data = pd.read_csv('test.csv', encoding = 'ISO-8859-1')
    
    # Load Address Data----------------------------------------------------------------------------------------------------
    
    addresses =  pd.read_csv('addresses.csv')
    lat_lons = pd.read_csv('latlons.csv')
    id_address= addresses.set_index('address').join(lat_lons.set_index('address')).reset_index().set_index('ticket_id')
    
    # Remove train only columns from train df----------------------------
    train_only_columns = ['payment_amount', 'payment_date', 'payment_status', 'balance_due','collection_status', 'compliance_detail']
    train_data.drop(train_only_columns, axis=1, inplace=True)
    
    # Drop inspector_name, violator_name, violation_street_name, mailing_address_str_name, zip_code,
    # violation_description from test and train----------------------------------------------------------------------------    
    cols_del=['inspector_name', 'violator_name', 'violation_street_name', 'mailing_address_str_name', 'zip_code', 'violation_description']
    train_data.drop(cols_del, axis=1, inplace=True)
    test_data.drop(cols_del, axis=1, inplace=True)

    ## Country********* 
    ### If the violator statying is within country USA or not, If the violator is statying within USA 1 else 0
    country_train_valcounts_=pd.DataFrame(train_data["country"].value_counts()).rename(columns={'country':'Total Rows'})
    country_test_valcounts_=pd.DataFrame(test_data["country"].value_counts()).rename(columns={'country':'Total Rows'})
    ### In train data set rows where country is USA we can change it to 1 or else 0, looks like there is only one country in the test data
    train_data = train_data[train_data.country == 'USA']
    train_data['USA'] = train_data['country'].apply(lambda x: 1 if 'USA' else 0)
    test_data['USA'] = test_data['country'].apply(lambda x: 1 if 'USA' else 0)

    ## City********* 
    ### If the violator is statying within city detriot, MI or not, and If the violator is statying within city detriot 1 else 0
    #train data
    train_data['within_Detriot'] = np.where((train_data.state == 'MI' ) & (train_data['city'].str.findall('det', flags=re.IGNORECASE)), 1, 0)
    #test data
    test_data['within_Detriot'] = np.where((test_data.state == 'MI' ) & (test_data['city'].str.findall('det', flags=re.IGNORECASE)), 1, 0)
   
    ## State********* 
    ### if the violator is statying within state, MI or not, if the violator is statying within state 1 else 0
    train_data['MI'] = np.where((train_data.state == 'MI' ), 1, 0)
    test_data['MI'] = np.where((test_data.state == 'MI' ), 1, 0)
    
    ## Drop country, city, state column from test and train data*************
    delete_columns=['country', 'city', 'state']
    train_data.drop(delete_columns, axis=1, inplace=True)
    test_data.drop(delete_columns, axis=1, inplace=True)
    
    ## Drop columns nan_columns**********
    # Pandas will recognize both empty cells and “NA” types as missing values. 
    # there are many empty rows in grafitti_status we can delete the column
    nan_columns=[ 'violation_street_number', 'violation_zip_code', 'mailing_address_str_number', 'non_us_str_code', 'violation_zip_code', 'non_us_str_code', 'grafitti_status']
    train_data.drop(nan_columns, axis=1, inplace=True)
    test_data.drop(nan_columns, axis=1, inplace=True)
    
    ## violation codes********* 
    ### get the top 10 violation codes and if the violation is in top 10 then 1 or else 0 for both test and train data.
    # train data 
    top10_violation_types = train_data.violation_code.value_counts().index[0:10]
    train_data['top10_violation_types'] = [1 if c in top10_violation_types else 0 for c in train_data.violation_code]
    # test data 
    test_data['top10_violation_types'] = [1 if c in top10_violation_types else 0 for c in test_data.violation_code]
    ## Drop violation_code from test and train data*************
    train_data.drop('violation_code', axis=1, inplace=True)
    test_data.drop('violation_code', axis=1, inplace=True)
    
    ## agency_name********* 
    agency_train_valcounts_=pd.DataFrame(train_data["agency_name"].value_counts()).rename(columns={'agency_name':'Total Rows'})
    agency_test_valcounts_=pd.DataFrame(test_data["agency_name"].value_counts()).rename(columns={'agency_name':'Total Rows'})
    agency_compliance_=pd.crosstab(train_data.agency_name, train_data.compliance)
    pd.merge(agency_compliance_, agency_train_valcounts_, left_index=True, right_index=True).reset_index().rename(columns={'index':'Agency_Name', 0.0:'Non_Compliant_0', 1.0:'Compliant_1'})    

    def agency_names():    
        fig, ax = plt.subplots(figsize=(10,10))
        color= ['dodgerblue', 'hotpink']
        sns.countplot(hue ='compliance', x = "agency_name", data = train_data, palette=color)
        plt.setp( ax.xaxis.get_majorticklabels(), rotation=-45, ha="left" )
        plt.tight_layout()
        return plt.show()
    
    delete_list=['Neighborhood City Halls']
    train_data = train_data[~train_data['agency_name'].isin(delete_list)]
    
    ## disposition********* 
    disposition_train_valcounts_=pd.DataFrame(train_data["disposition"].value_counts()).rename(columns={'disposition':'Total Rows'})
    disposition_test_valcounts_=pd.DataFrame(test_data["disposition"].value_counts()).rename(columns={'disposition':'Total Rows'})
    ### In the train data there are some more columns where the disposition is under not responsible, we can delete the     
    disposition_compliance_=pd.crosstab(train_data.disposition, train_data.compliance)
    delete_list=['Not responsible by Dismissal', 'Not responsible by City Dismissal', 'Not responsible by Determination',
                 'PENDING JUDGMENT','SET-ASIDE (PENDING JUDGMENT)']
    train_data = train_data[~train_data['disposition'].isin(delete_list)]    
    
    # Relation between numeric variables-------------------------------------------------------------------------
    ### state_fee and admin_fee is same irrespective of compliance hence-ignore those columns
    ### clean_up_cost is mostly 0-ignore the column
    ### discount_amount there are very few rows with discount amount-ignore that column
    ### judgment_amount is sum of fine amount and other fee-ignore that column    
    
    def fine_late_fees():
        sns.scatterplot(data=test_data, x="fine_amount", y="late_fee")
        corr_val = test_data[['fine_amount','late_fee']].corr()
        return corr_val, plt.show() 
    ### there is high correlation between the fine_amount and late fee, based on the correlation either of 
    ### fine_amount or late_fee will be used for the model building
    
    # convert the dates to usable manner--------------------------------------------------------------------------------------    
    train_data[['ticket_issued_date','hearing_date']] = train_data[['ticket_issued_date','hearing_date']].apply(pd.to_datetime)
    test_data[['ticket_issued_date','hearing_date']] = test_data[['ticket_issued_date','hearing_date']].apply(pd.to_datetime) 

    # forward fill to fill the gaps in hearing date
    train_data.hearing_date.fillna(method='ffill', inplace=True)
    test_data.hearing_date.fillna(method='ffill', inplace=True)    

    train_data['ticket_hearing_gap'] = (train_data['hearing_date'] - train_data['ticket_issued_date']).dt.days    
    ticket_hearing_gap_agg = train_data.groupby('compliance').ticket_hearing_gap.agg(['mean','median','std','var']).reset_index()
    test_data['ticket_hearing_gap'] = (test_data['hearing_date'] - test_data['ticket_issued_date']).dt.days    
    delete_columns=['clean_up_cost' ,'judgment_amount', 'late_fee', 'ticket_issued_date', 'hearing_date']
    train_data.drop(delete_columns, axis=1, inplace=True)
    test_data.drop(delete_columns, axis=1, inplace=True)    
    train_data.reset_index(drop=True, inplace=True)
    test_data.reset_index(drop=True, inplace=True)    
    
    # Create Dummies----------------------------------------------------------------------------------------------------------
    # One-Hot encoding the categorical parameters using get_dummies()
    training_df = pd.get_dummies(train_data, columns = ['agency_name', 'disposition'])
    testing_df = pd.get_dummies(test_data, columns = ['agency_name', 'disposition'])
    train_ticketid=training_df[['ticket_id']]
    test_ticketid=testing_df[['ticket_id']]
    
    training_df.drop(['ticket_id', 'agency_name_Health Department'], axis=1, inplace=True)
    delete_columns=['ticket_id',
                    'disposition_Responsible (Fine Waived) by Admis',
                    'disposition_Responsible - Compl/Adj by Default',
                    'disposition_Responsible - Compl/Adj by Determi',
                    'disposition_Responsible by Dismissal']

    testing_df.drop(delete_columns, axis=1, inplace=True)
    
    ### rename columns

    col_name={'agency_name_Buildings, Safety Engineering & Env Department': 'Agency_BSE',
              'agency_name_Department of Public Works': 'Agency_DPW',
              'agency_name_Detroit Police Department': 'Agency_DPD',
              'disposition_Responsible (Fine Waived) by Deter': 'Dis_res_deter',
              'disposition_Responsible by Admission':'Dis_res_admission',
              'disposition_Responsible by Default':'Dis_res_default',
              'disposition_Responsible by Determination':'Dis_res_determination'}

    training_df.rename(columns=col_name, inplace=True)
    testing_df.rename(columns=col_name, inplace=True)

    # Handling class imbalance-------------------------------------------------------------------------
    compliance_=pd.DataFrame(training_df["compliance"].value_counts().apply(lambda x: x/len(training_df["compliance"]))).rename(columns={'compliance':'percentage of class'}).T
    ### There is a huge difference between the class of target variables. 148273 non-compliant transactions and 11595 compliant. 

    X = training_df.drop('compliance', axis=1)
    y = training_df.compliance

    # Test train split
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    
    # Support Vector Machine Python
    clf_model = RandomForestClassifier(min_samples_split=100, max_depth=5, random_state=0,class_weight={0:13, 1:1}).fit(X_train, y_train)

    #get predictions
    y_pred = clf_model.predict(X_test)

    # predict probabilities
    y_probs = clf_model.predict_proba(X_test)

    # keep probabilities for the positive outcome only
    y_probs = y_probs[:, 1]

    # calculate roc curve
    fpr_, tpr_, _ = roc_curve(y_test, y_probs)

    # calculate AUC
    auc_ = roc_auc_score(y_test, y_probs)

    #confusion matrix
    cm=confusion_matrix(y_test,y_pred)

    specificity_ = cm[0,0]/(cm[0,0]+cm[0,1]) #TN/(TN+FP)
    
    #print_this= print("auc for the Classifier model is {auc}".format(auc=auc_)) 

    # get probability for testing df
    probability = clf_model.predict_proba(testing_df)[:,1]
    # we need to select the probability that the ticket will be paid
    final_result=pd.Series(probability, index =list(test_ticketid['ticket_id']))

    return final_result# Your answer here